In [28]:
import pandas as pd
import numpy as np
import json 


### Trier les categories

In [29]:
# create an empty DataFrame with the columns 'dep', 'year', 'month', 'type', 'count'
df2 = pd.DataFrame(columns=['dep', 'year', 'month', 'type', 'count'])

# recup the name of a sheet of a excel file
file_path = 'data/crime/1996.xlsx'
excel_file = pd.ExcelFile(file_path)
sheet_name = excel_file.sheet_names[2]

#show data of sheet_name
df = pd.read_excel(file_path, sheet_name=sheet_name)
df



,"Tableau 1. Faits constatés par la police et la gendarmerie, par département de France métropolitaine, en janvier 1996.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Index de l'Etat 4001,Ain,Aisne,Allier,Alpes-de-Haute-Provence,Hautes-Alpes,Alpes-Maritimes,Ardèche,Ardennes,Ariège,...,Vienne,Haute-Vienne,Vosges,Yonne,Territoire de Belfort,Essonne,Hauts-de-Seine,Seine-Saint-Denis,Val-de-Marne,Val-d'Oise
2,Règlements de compte entre malfaiteurs,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Homicides pour voler et à l'occasion de vols,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Homicides pour d'autres motifs,0,1,0,0,0,4,0,0,0,...,0,0,0,0,1,2,1,5,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Infractions au droit de l'urbanisme et de la c...,0,0,0,2,0,3,0,1,3,...,1,1,1,0,0,4,7,0,6,1
102,Fraudes fiscales,0,2,0,0,1,2,0,0,0,...,0,1,0,0,0,0,0,1,1,0
103,Autres délits économiques et financiers,1,0,3,0,1,0,0,0,0,...,0,1,0,0,0,2,10,7,4,0
104,Autres délits,9,17,2,10,4,30,4,4,10,...,21,25,13,6,8,41,52,83,54,54


In [30]:
infractions = np.array(df["Tableau 1. Faits constatés par la police et la gendarmerie, par département de France métropolitaine, en janvier 1996."][2:].to_list()[:-1])
infractions

array(['Règlements de compte entre malfaiteurs',
       "Homicides pour voler et à l'occasion de vols",
       "Homicides pour d'autres motifs",
       "Tentatives d'homicides pour voler et à l'occasion de vols",
       "Tentatives homicides pour d'autres motifs",
       'Coups et blessures volontaires suivis de mort',
       'Autres coups et blessures volontaires criminels ou correctionnels',
       "Prises d'otages à l'occasion de vols",
       "Prises d'otages dans un autre but", 'Séquestrations',
       'Menaces ou chantages pour extorsion de fonds',
       'Menaces ou chantages dans un autre but',
       'Atteintes à la dignité et à la  personnalité',
       'Violations de domicile',
       'Vols à main armée contre des établissements financiers',
       'Vols à main armée contre des établissements industriels ou commerciaux',
       'Vols à main armée contre des entreprises de transports de fonds',
       'Vols à main armée contre des particuliers à leur domicile',
       'Autres

In [31]:
def get_all(keyword, list_of_infractions):
    res = []
    for word in list_of_infractions:
        if keyword.lower() in str(word).lower():
            res.append(word)
            list_of_infractions = list_of_infractions[list_of_infractions != word]
    res = np.array(res)
    res = res.astype(str)
    return res, list_of_infractions


def clean_selection(category, values_to_remove, infractions):
    category = category[~np.isin(category, values_to_remove)]
    np_values_to_remove = np.array(values_to_remove)
    infractions = np.concatenate((infractions, np_values_to_remove))
    return category, infractions


def force_add(category, values_to_add, infractions):
    infractions = infractions[~np.isin(infractions, values_to_add)]
    np_values_to_add = np.array(values_to_add)
    category = np.concatenate((category, np_values_to_add))
    return category, infractions


def get_category(keyword, values_to_remove, infractions, values_to_add):
    category, infractions = get_all(keyword, infractions)
    category, infractions = force_add(category, values_to_add, infractions)
    return clean_selection(category, values_to_remove, infractions)


def complete_category(category_to_complete, infractions, keyword):
    new_category, infractions = get_all(keyword, infractions)
    new_category = np.array(new_category)
    category_to_complete = np.concatenate((category_to_complete, new_category))
    return category_to_complete, infractions


def get_complete_category(keywords, list_of_infractions, values_to_remove, values_to_add):
    category, infractions = get_all(keywords[0], list_of_infractions)
    if len(keywords) > 1:
        for word in keywords[1:]:
            category, infractions = complete_category(category, infractions, word)
    category, infractions = force_add(category, values_to_add, infractions)
    return clean_selection(category, values_to_remove, infractions)



In [32]:
meurtre, infractions = get_complete_category(["homici", "mort"], infractions, [], [])
print(meurtre)

["Homicides pour voler et à l'occasion de vols"
 "Homicides pour d'autres motifs"
 "Tentatives d'homicides pour voler et à l'occasion de vols"
 "Tentatives homicides pour d'autres motifs"
 'Homicides commis contre enfants de moins de 15 ans'
 'Coups et blessures volontaires suivis de mort']


In [33]:
vol, infractions = get_complete_category(["vol", "cambr"], infractions, [
    "Coups et blessures volontaires suivis de mort",
    "Autres coups et blessures volontaires criminels ou correctionnels",
    "Incendies volontaires de biens publics",
    "Incendies volontaires de biens privés",
    
    ], [])
print(vol)

["Prises d'otages à l'occasion de vols"
 'Vols à main armée contre des établissements financiers'
 'Vols à main armée contre des établissements industriels ou commerciaux'
 'Vols à main armée contre des entreprises de transports de fonds'
 'Vols à main armée contre des particuliers à leur domicile'
 'Autres vols à main armée'
 'Vols avec armes blanches contre des établissements financiers, commerciaux ou industriels'
 'Vols avec armes blanches contre des particuliers à leur domicile'
 'Autres vols avec armes blanches'
 'Vols violents sans arme contre des établissements financiers, commerciaux ou industriels'
 'Vols violents sans arme contre des particuliers à leur domicile'
 'Vols violents sans arme contre des femmes sur voie publique ou autre lieu public'
 "Vols violents sans arme contre d'autres victimes"
 'Vols avec entrée par ruse en tous lieux' 'Vols à la tire'
 "Vols à l'étalage" 'Vols de véhicules de transport avec fret'
 "Vols d'automobiles" 'Vols de véhicules motorisés à 2 rou

In [34]:
crime_sexuel, infractions = get_complete_category(["viols", "sex", "Proxé"], infractions, [], [])
print(crime_sexuel)

['Viols sur des majeur(e)s' 'Viols sur des mineur(e)s'
 'Harcèlements sexuels et autres agressions sexuelles contre des majeur(e)s'
 'Harcèlements sexuels et autres agressions sexuelles contre des mineur(e)s'
 'Atteintes sexuelles' 'Proxénétisme']


In [35]:
drogue, infractions = get_complete_category(["stup"], infractions, [], [])
print(drogue)

['Trafic et revente sans usage de stupéfiants'
 'Usage-revente de stupéfiants' 'Usage de stupéfiants'
 'Autres infractions à la législation sur les stupéfiants']


In [36]:
violence, infractions = get_complete_category([
    "menac", 
    "violen", 
    "Outrage", 
    "anima", 
    "Séquestration", 
    "otage",
    "Règlements de compte",
    "coup",
    "dignité"
    ], infractions, [], [])
print(violence)

['Menaces ou chantages pour extorsion de fonds'
 'Menaces ou chantages dans un autre but'
 "Violences, mauvais traitements et abandons d'enfants."
 'Violences à dépositaires autorité' 'Outrages à dépositaires autorité'
 'Destructions, cruautés et autres délits envers les animaux'
 'Séquestrations' "Prises d'otages dans un autre but"
 'Règlements de compte entre malfaiteurs'
 'Coups et blessures volontaires suivis de mort'
 'Autres coups et blessures volontaires criminels ou correctionnels'
 'Atteintes à la dignité et à la  personnalité']


In [37]:
fraude, infractions = get_complete_category([
    "fraude", 
    "recel", 
    "boisson", 
    "jeux", 
    "alimentaire", 
    "étranger",
    "fau",
    "Falsifi",
    "contrefaçon",
    "facture",
    "garde",
    "abus de",
    "financiers",
    "Marchandage",
    "réglement",
    "travail",
    "législation",
    "Infraction"
    ], infractions, [], [])
print(fraude)

["Fraudes alimentaires et infractions à l'hygiène"
 'Contrefaçons et fraudes industrielles et commerciales'
 'Fraudes fiscales' 'Recels'
 "Délits de débits de boissons et infraction à la réglementation sur l'alcool  et le tabac"
 'Délits des courses et des jeux' 'Non versement de pension alimentaire'
 "Infractions aux conditions générales d'entrée et de séjour des étrangers"
 "Aide à l'entrée, à la circulation et au séjour des étrangers"
 'Autres infractions à la police des étrangers'
 "Emploi d'étranger sans titre de travail" "Faux documents d'identité"
 'Faux documents concernant la circulation des véhicules'
 'Autres faux documents administratifs'
 'Faux en écriture publique et authentique' 'Autres faux en écriture'
 'Fausse monnaie'
 'Prix illicites, publicité fausse et infractions aux règles de la concurrence'
 'Falsification et usages de cartes de crédit'
 'Contrefaçons littéraires et artistique' 'Achats et ventes sans factures'
 'Délits au sujet de la garde des mineurs'
 'Escroq

In [38]:
vandalisme, infractions = get_complete_category(["destruction", "environnement", "chasse", "biens p"], infractions, [], [])
print(vandalisme)

['Autres destructions et dégradations de biens publics'
 'Autres destructions et dégradations de biens privés'
 'Destructions et dégradations de véhicules privés'
 "Atteintes à l'environnement" 'Chasse et pêche'
 "Attentats à l'explosif contre des biens publics"
 "Attentats à l'explosif contre des biens privés"
 'Incendies volontaires de biens publics'
 'Incendies volontaires de biens privés']


In [39]:
autre, infractions = get_complete_category([""], infractions, [], [])
print(autre)

['Violations de domicile' 'Port ou détention armes prohibées'
 'Atteintes aux intérêts fondamentaux de la Nation'
 'Délits interdiction de séjour et de paraître' 'Autres délits']


In [40]:
categories = {"meurtre" : meurtre.tolist(),
              "vol" : vol.tolist(),
              "crime_sexuel" : crime_sexuel.tolist(),
              "drogue" : drogue.tolist(),
              "violence" : violence.tolist(),
              "fraude" : fraude.tolist(),
              "vandalisme" : vandalisme.tolist(),
              "autre" : autre.tolist()}

json_file_path = "data/crime/crimes_category.json"

with open(json_file_path, 'w') as jsonfile:
    json.dump(categories, jsonfile, ensure_ascii=False)

categories

{'meurtre': ["Homicides pour voler et à l'occasion de vols",
  "Homicides pour d'autres motifs",
  "Tentatives d'homicides pour voler et à l'occasion de vols",
  "Tentatives homicides pour d'autres motifs",
  'Homicides commis contre enfants de moins de 15 ans',
  'Coups et blessures volontaires suivis de mort'],
 'vol': ["Prises d'otages à l'occasion de vols",
  'Vols à main armée contre des établissements financiers',
  'Vols à main armée contre des établissements industriels ou commerciaux',
  'Vols à main armée contre des entreprises de transports de fonds',
  'Vols à main armée contre des particuliers à leur domicile',
  'Autres vols à main armée',
  'Vols avec armes blanches contre des établissements financiers, commerciaux ou industriels',
  'Vols avec armes blanches contre des particuliers à leur domicile',
  'Autres vols avec armes blanches',
  'Vols violents sans arme contre des établissements financiers, commerciaux ou industriels',
  'Vols violents sans arme contre des part